In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from functools import reduce

In [2]:
df_all_services = pd.read_csv('../data/df_all_services.csv')

In [ ]:
df_all_services.head()

# Test 1
#### Using one function to get both values

In [3]:
df_test = df_all_services.head(30)
df_test.head(35)

,title,tv,year,rating,score_imdb,score_rotten,netflix,Hulu,Prime Video,Disney+,HBO Max,link
0,Breaking Bad,TV,2008,18+,9.5,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/show/breaking-bad-2008
1,Inception,NaN,2010,13+,8.8,87%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/inception-2010
2,Back to the Future,NaN,1985,7+,8.5,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/back-to-the-future-...
3,The Matrix,NaN,1999,18+,8.7,88%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/the-matrix-1999
4,Stranger Things,TV,2016,16+,8.8,93%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/show/stranger-things-2016
5,The Silence of the Lambs,NaN,1991,18+,8.6,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/the-silence-of-the-...
6,Avengers: Infinity War,NaN,2018,13+,8.5,85%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/avengers-infinity-w...
7,"The Good, the Bad and the Ugly",NaN,1966,18+,8.8,97%,Yes,NaN,Yes,NaN,NaN,https://reelgood.com/movie/the-good-the-bad-an...
8,Django Unchained,NaN,2012,18+,8.4,87%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/django-unchained-2012
9,The Pianist,NaN,2002,18+,8.5,95%,Yes,NaN,Yes,NaN,NaN,https://reelgood.com/movie/the-pianist-2002


In [4]:
#bringing in the function written in the genre data prework notebook.

def get_genres_and_country(link):
    '''
    This function uses a link to detailed info for a title in reelgood.com to return a dictionary
    with values for genres and country
    '''
    details_soup = BeautifulSoup(requests.get(link).content, 'lxml') #get the soup out of the link
    details_section = details_soup.find_all('div',attrs={'class': 'css-1ss0qk ey4ir3j0'}) #select the section which includes the details for the title
    details_section_soup = BeautifulSoup(str(details_section),'lxml') #turn the details section into soup
    genre_section = details_section_soup.find_all('a',attrs={'href': re.compile('/(movies|tv)/genre/')}) #find the genres
    #add the text into a list
    #first two links are duplicated in "more genres", so skipping them in the range
    genres = []
    
    for i in range (0,len(genre_section)): 
        genres.append(genre_section[i].text)
        genres = list(dict.fromkeys(genres)) #removing duplicates, as sometimes duplicates repeated in "more genres"
    
    genres_str = ', '.join(map(str,genres))#turn list of genres into one string
    
    #starting here, code to get the country
    country_section = details_section_soup.find_all('a', attrs={'href':re.compile('^/country/')})
    country = country_section[0].text
    return {'genres' : genres_str, 'country': country}

In [5]:
df_test['genres'] = ''
df_test['country'] = ''
df_test.head(2)

C:\Users\diego\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\diego\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,tv,year,rating,score_imdb,score_rotten,netflix,Hulu,Prime Video,Disney+,HBO Max,link,genres,country
0,Breaking Bad,TV,2008,18+,9.5,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/show/breaking-bad-2008,,
1,Inception,NaN,2010,13+,8.8,87%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/inception-2010,,


In [6]:
%%time
#run function in our combined dataframe
for row in df_test.itertuples():
    try:
        details = get_genres_and_country(row.link)
        df_test.at[row.Index,'genres'] = details['genres']
        df_test.at[row.Index,'country'] = details['country']
    except:
        continue

Wall time: 2min 41s


In [7]:
df_test

,title,tv,year,rating,score_imdb,score_rotten,netflix,Hulu,Prime Video,Disney+,HBO Max,link,genres,country
0,Breaking Bad,TV,2008,18+,9.5,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/show/breaking-bad-2008,"Crime, Drama, Thriller",America
1,Inception,NaN,2010,13+,8.8,87%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/inception-2010,"Action & Adventure, Mystery, Science-Fiction, ...",United Kingdom
2,Back to the Future,NaN,1985,7+,8.5,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/back-to-the-future-...,"Comedy, Fantasy, Science-Fiction, Action & Adv...",America
3,The Matrix,NaN,1999,18+,8.7,88%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/the-matrix-1999,,Australia
4,Stranger Things,TV,2016,16+,8.8,93%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/show/stranger-things-2016,"Drama, Fantasy, Horror, Mystery, Science-Ficti...",America
5,The Silence of the Lambs,NaN,1991,18+,8.6,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/the-silence-of-the-...,"Drama, Mystery, Thriller, Crime, Horror",America
6,Avengers: Infinity War,NaN,2018,13+,8.5,85%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/avengers-infinity-w...,"Action & Adventure, Fantasy, Science-Fiction",America
7,"The Good, the Bad and the Ugly",NaN,1966,18+,8.8,97%,Yes,NaN,Yes,NaN,NaN,https://reelgood.com/movie/the-good-the-bad-an...,,Germany
8,Django Unchained,NaN,2012,18+,8.4,87%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/django-unchained-2012,,America
9,The Pianist,NaN,2002,18+,8.5,95%,Yes,NaN,Yes,NaN,NaN,https://reelgood.com/movie/the-pianist-2002,,France


# Test 2

#### Using separate function for country and genres.

In [9]:
df_test2 = df_all_services.head(30)

In [10]:
df_test['genres'] = ''
df_test['country'] = ''
df_test.head(2)

C:\Users\diego\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\diego\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,tv,year,rating,score_imdb,score_rotten,netflix,Hulu,Prime Video,Disney+,HBO Max,link,genres,country
0,Breaking Bad,TV,2008,18+,9.5,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/show/breaking-bad-2008,,
1,Inception,NaN,2010,13+,8.8,87%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/inception-2010,,


In [11]:
def get_genres(link):
    details_soup = BeautifulSoup(requests.get(link).content, 'lxml') #get the soup out of the link
    details_section = details_soup.find_all('div',attrs={'class': 'css-1ss0qk ey4ir3j0'}) #select the section which includes the details for the title
    details_section_soup = BeautifulSoup(str(details_section),'lxml') #turn the details section into soup
    genre_section = details_section_soup.find_all('a',attrs={'href': re.compile('/(movies|tv)/genre/')}) #find the genres
    #add the text into a list
    genres = []
    
    for i in range (0,len(genre_section)): 
        genres.append(genre_section[i].text)
        genres = list(dict.fromkeys(genres)) #removing duplicates, as sometimes duplicates repeated in "more genres"
    
    genres_str = ', '.join(map(str,genres))#turn list of genres into one string
    
    return genres_str

In [12]:
%%time
df_test2['genres'] = df_test2.link.apply(get_genres)


Wall time: 2min 35s


C:\Users\diego\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
df_test2

,title,tv,year,rating,score_imdb,score_rotten,netflix,Hulu,Prime Video,Disney+,HBO Max,link,genres
0,Breaking Bad,TV,2008,18+,9.5,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/show/breaking-bad-2008,"Crime, Drama, Thriller"
1,Inception,NaN,2010,13+,8.8,87%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/inception-2010,"Action & Adventure, Mystery, Science-Fiction, ..."
2,Back to the Future,NaN,1985,7+,8.5,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/back-to-the-future-...,"Comedy, Fantasy, Science-Fiction, Action & Adv..."
3,The Matrix,NaN,1999,18+,8.7,88%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/the-matrix-1999,"Action & Adventure, Science-Fiction"
4,Stranger Things,TV,2016,16+,8.8,93%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/show/stranger-things-2016,"Drama, Fantasy, Horror, Mystery, Science-Ficti..."
5,The Silence of the Lambs,NaN,1991,18+,8.6,96%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/the-silence-of-the-...,"Drama, Mystery, Thriller, Crime, Horror"
6,Avengers: Infinity War,NaN,2018,13+,8.5,85%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/avengers-infinity-w...,"Action & Adventure, Fantasy, Science-Fiction"
7,"The Good, the Bad and the Ugly",NaN,1966,18+,8.8,97%,Yes,NaN,Yes,NaN,NaN,https://reelgood.com/movie/the-good-the-bad-an...,
8,Django Unchained,NaN,2012,18+,8.4,87%,Yes,NaN,NaN,NaN,NaN,https://reelgood.com/movie/django-unchained-2012,Drama
9,The Pianist,NaN,2002,18+,8.5,95%,Yes,NaN,Yes,NaN,NaN,https://reelgood.com/movie/the-pianist-2002,"Biography, Drama"


### getting both genre and country seems to be faster. 

In [8]:
#export test data to play with in tableau 

df_test.to_csv('../data/df_test.csv')